In [85]:
import numpy as np
import time

class LinearRegression:
    def __init__(self):
        self.theta = None

    def fit(self, X, y):
        # Add the bias term
        X_b = np.c_[np.ones((X.shape[0], 1)), X]
        # Compute the matrix A (X^T X) and the vector b (X^T y)
        A = X_b.T.dot(X_b)
        b = X_b.T.dot(y)
        # Solve the linear equation A * theta = b for theta
        self.theta = np.linalg.solve(A, b)

    def predict(self, X):
        # Add bias term 
        X_b = np.c_[np.ones((X.shape[0], 1)), X]
        return X_b.dot(self.theta)


class LogisticRegressionGD:
    def __init__(self, learning_rate=0.01, epochs=1000, method="gd", tol=1e-4):
        self.duration = None
        self.total_iterations = None
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.method = method
        self.tol = tol  # tolerance for stopping condition
        self.theta = None

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def cost_function(self, X, y):
        m = len(y)
        predictions = self.sigmoid(X.dot(self.theta))
        cost = -1/m * (y.T.dot(np.log(predictions)) + (1 - y).T.dot(np.log(1 - predictions)))
        return cost[0][0]

    def predict_proba(self, X):
        probabilities = self.sigmoid(X.dot(self.theta))
        return probabilities


    def fit(self, X, y):
        m, n = X.shape
        self.theta = np.zeros((n, 1))
        prev_cost = float('inf')

        # Initialize the total iterations counter
        total_iterations = 0

        # Record the start time
        start_time = time.time()

        for epoch in range(self.epochs):
            if self.method == "gd":  # Gradient Descent
                gradient = X.T.dot(self.sigmoid(X.dot(self.theta)) - y) / m
                self.theta -= self.learning_rate * gradient
                total_iterations += 1
            elif self.method == "sgd":  # Stochastic Gradient Descent
                for i in range(m):
                    random_index = np.random.randint(m)
                    xi = X[random_index:random_index+1]
                    yi = y[random_index:random_index+1]
                    gradient = xi.T.dot(self.sigmoid(xi.dot(self.theta)) - yi)
                    self.theta -= self.learning_rate * gradient
                    total_iterations += 1

            # Calculate cost for current epoch
            current_cost = self.cost_function(X, y)

            # Check stopping condition
            if abs(prev_cost - current_cost) < self.tol:
                # Calculate the time taken
                duration = time.time() - start_time
                print(f"{self.method.title()} converged after {total_iterations} iterations in {duration:.4f} seconds. Stopping.")
                break

            prev_cost = current_cost

        # If the loop finishes without convergence
        if self.method == "gd" and epoch == self.epochs - 1:
            duration = time.time() - start_time
            print(f"Gradient Descent completed {self.epochs} epochs in {duration:.4f} seconds.")

        if self.method == "sgd" and epoch == self.epochs - 1:
            duration = time.time() - start_time
            print(f"Stochastic Gradient Descent completed {total_iterations} iterations in {duration:.4f} seconds.")

        # Store the total iterations and duration in the class instance for later reference
        self.total_iterations = total_iterations
        self.duration = duration

        return self

In [86]:
import pandas as pd

# Load the data from the Training tab
training_data = pd.read_excel('Project3.xlsx', sheet_name='Training', engine='openpyxl')

# Load the data from the Predict tab
predict_data = pd.read_excel('Project3.xlsx', sheet_name='Predict', engine='openpyxl')


In [87]:
#Step 1(a)

X_train = training_data[['Midterm', 'Homework', 'Quiz']].values
y_train = training_data['Course Grade'].values.reshape(-1, 1)

# Normalize the features to improve convergence
X_train_mean = np.mean(X_train, axis=0)
X_train_std = np.std(X_train, axis=0)
X_train = (X_train - X_train_mean) / X_train_std

X_train_bias = np.c_[np.ones((X_train.shape[0], 1)), X_train]

# Define the LinearRegression class and fit the model
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

# Display the weight vector
print("Weight vector (theta):", lin_reg.theta)

Weight vector (theta): [[79.835     ]
 [ 4.17743519]
 [ 3.03946093]
 [ 6.71839328]]


In [88]:
#Step 1(b)

X_predict = predict_data[['Midterm', 'Homework', 'Quiz']].values

# Normalize using the mean and std from the training data
X_predict = (X_predict - X_train_mean) / X_train_std

X_predict_bias = np.c_[np.ones((X_predict.shape[0], 1)), X_predict]

# Use the model to make predictions
y_pred = lin_reg.predict(X_predict)

# Add predictions to the predict_data dataframe
predict_data['Estimated Course Grade'] = y_pred.flatten()
predict_data_sgd = predict_data

# Display the predictions
print(predict_data[['Midterm', 'Homework', 'Quiz', 'Estimated Course Grade']])


    Midterm  Homework  Quiz  Estimated Course Grade
0        49        79    71               65.243032
1        56        21    46               48.356620
2        58       100    79               74.881560
3        61        82    58               64.225981
4        62        90    62               67.470391
5        66        99    75               75.770889
6        71        73    65               69.362745
7        73        87    80               78.570618
8        73        97    73               77.029189
9        74        73    77               75.563111
10       74        85    66               72.594112
11       74       100    87               83.842523
12       78        79    56               68.785211
13       80        72    71               74.901825
14       80        82    90               84.553235
15       81        56    60               68.155962
16       81        80    78               79.437717
17       82       100    79               83.157079
18       83 

In [89]:
# Step 2(a)

# Convert course grades to binary labels
training_data['Course Grade'] = training_data['Course Grade'].apply(lambda x: 0 if x < 70 else 1)

y_train = training_data['Course Grade'].values.reshape(-1, 1)

# Display the modified labels
print(training_data[['Midterm', 'Homework', 'Quiz', 'Course Grade']])


     Midterm  Homework  Quiz  Course Grade
0         37        23    41             0
1         38       100    54             0
2         40        16    24             0
3         44        93    46             0
4         48        96    39             0
..       ...       ...   ...           ...
395      100        70    84             1
396      100        81    94             1
397      100        96   100             1
398      100        99    93             1
399      100       100   100             1

[400 rows x 4 columns]


In [90]:
# Step 2(b)


# Create an instance of LogisticRegressionGD (SGD) and fit the training data
log_reg_sgd = LogisticRegressionGD(learning_rate=0.001, epochs=10000, method="sgd")
log_reg_sgd.fit(X_train_bias, y_train)
print(log_reg_sgd.theta)

# Create an instance of LogisticRegressionGD (GD) and fit the training data
log_reg = LogisticRegressionGD(learning_rate=0.001, epochs=10000, method="gd")
log_reg.fit(X_train_bias, y_train)
print(log_reg.theta)


Sgd converged after 48800 iterations in 0.6354 seconds. Stopping.
[[2.86754413]
 [0.85101467]
 [0.89257833]
 [1.22435293]]
Gd converged after 1310 iterations in 0.0428 seconds. Stopping.
[[0.38972032]
 [0.16213183]
 [0.17888972]
 [0.22566503]]


In [91]:
# Use the trained model to estimate the probability of passing
probabilities_passing = log_reg.predict_proba(X_predict_bias)
probabilities_passing_sgd = log_reg_sgd.predict_proba(X_predict_bias)


# Add probabilities to the predict_data dataframe
predict_data['Probability of Passing'] = probabilities_passing
predict_data_sgd['Probability of Passing'] = probabilities_passing_sgd


# Display the data along with the predicted probabilities
print(f"\nGradient Descent Predicted Data: ")
print(predict_data[['Midterm', 'Homework', 'Quiz', 'Probability of Passing']])
print(f"\nStochastic Gradient Descent Predicted Data: ")
print(predict_data_sgd[['Midterm', 'Homework', 'Quiz', 'Probability of Passing']])




Gradient Descent Predicted Data: 
    Midterm  Homework  Quiz  Probability of Passing
0        49        79    71                0.484883
1        56        21    46                0.017339
2        58       100    79                0.891593
3        61        82    58                0.473087
4        62        90    62                0.650701
5        66        99    75                0.909869
6        71        73    65                0.680224
7        73        87    80                0.935672
8        73        97    73                0.928268
9        74        73    77                0.870678
10       74        85    66                0.826744
11       74       100    87                0.979339
12       78        79    56                0.689933
13       80        72    71                0.859904
14       80        82    90                0.976737
15       81        56    60                0.581785
16       81        80    78                0.941570
17       82       100    79  

In [92]:
print(predict_data.compare(predict_data_sgd))

Empty DataFrame
Columns: []
Index: []
